In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [9]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT ROUND(AVG(price), 2) FROM home_sales
WHERE bedrooms == 4
GROUP BY YEAR(date)
""").show()


+--------------------+
|round(avg(price), 2)|
+--------------------+
|           296363.88|
|            300263.7|
|           298353.78|
|           301819.44|
+--------------------+



In [11]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT ROUND(AVG(price), 2) FROM home_sales
WHERE bedrooms == 3 AND bathrooms == 3
GROUP BY date_built
""").show()


+--------------------+
|round(avg(price), 2)|
+--------------------+
|           290555.07|
|           293683.19|
|           292676.79|
|           290852.27|
|           295962.27|
|           291117.47|
|            288770.3|
|           292859.62|
+--------------------+



In [12]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT ROUND(AVG(price), 2) FROM home_sales
WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living > 2000
GROUP BY date_built
""").show()


+--------------------+
|round(avg(price), 2)|
+--------------------+
|            293965.1|
|           307539.97|
|           280527.62|
|           297619.46|
|           303676.79|
|           276553.81|
|           297609.97|
|           285010.22|
+--------------------+



In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
SELECT ROUND(AVG(price), 2) FROM home_sales
WHERE price > 350000
GROUP BY view
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+
|round(avg(price), 2)|
+--------------------+
|           403005.77|
|           788128.21|
|            404673.3|
|           798684.82|
|           399548.12|
|           397771.65|
|           750537.94|
|            396964.5|
|           1072285.2|
|           752861.18|
|           767036.67|
|            398867.6|
|            397862.0|
|           401419.75|
|            791453.0|
|           398592.71|
|           402124.62|
|           402022.68|
|          1056336.74|
|           399586.53|
+--------------------+
only showing top 20 rows

--- 1.991065263748169 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
# same query as above
spark.sql("""SELECT ROUND(AVG(price), 2) FROM home_sales
WHERE price > 350000
GROUP BY view""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+
|round(avg(price), 2)|
+--------------------+
|           403005.77|
|           788128.21|
|            404673.3|
|           798684.82|
|           399548.12|
|           397771.65|
|           750537.94|
|            396964.5|
|           1072285.2|
|           752861.18|
|           767036.67|
|            398867.6|
|            397862.0|
|           401419.75|
|            791453.0|
|           398592.71|
|           402124.62|
|           402022.68|
|          1056336.74|
|           399586.53|
+--------------------+
only showing top 20 rows

--- 0.6396448612213135 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data


In [ ]:
# 11. Read the parquet formatted data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False